- See Github repository for all original data files. 
- All manipulated data files required for this notebook are in folder "ManipulatedData", unzip the folder before executing.

In [1]:
# import packages
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt

# 1. Daily COVID data 
- required for forecasting and Bollinger Bands analysis.

In [2]:
# import data
ny_covid = pd.read_csv("ManipulatedData/NYC_covid.csv")
la_covid = pd.read_csv("ManipulatedData/LA_county_covid.csv")

# data cleaning & indexing
ny_covid['date_of_interest'] = ny_covid['date_of_interest'].apply(pd.to_datetime, utc=True)
la_covid['date_use'] = la_covid['date_use'].apply(pd.to_datetime, utc=True)
ny_covid = ny_covid.set_index(['date_of_interest'])
la_covid = la_covid.set_index(['date_use'])

# create combined file
coviddata = ny_covid[["CASE_COUNT", "DEATH_COUNT"]]
coviddata["LACase"] = la_covid["new_case"][:-1]
coviddata["LADeath"] = la_covid["new_deaths"][:-1]
coviddata = coviddata.rename(columns={"CASE_COUNT": "NYCase", "DEATH_COUNT": "NYDeath"})

In [3]:
coviddata

,NYCase,NYDeath,LACase,LADeath
date_of_interest,,,,
2020-03-01 00:00:00+00:00,0,0,25,0
2020-03-02 00:00:00+00:00,0,0,19,0
2020-03-03 00:00:00+00:00,1,0,13,0
2020-03-04 00:00:00+00:00,5,0,19,0
2020-03-05 00:00:00+00:00,3,0,33,0
...,...,...,...,...
2020-11-14 00:00:00+00:00,931,10,2563,11
2020-11-15 00:00:00+00:00,795,12,1518,14
2020-11-16 00:00:00+00:00,1419,8,4205,18


In [4]:
coviddata.to_csv("coviddata.csv", sep=",")

# 2. Prepare COVID, Crime, Economy Monthly Data
## 2.1 Helper function

In [5]:
# initialize time frame for final dataset
final_data = pd.DataFrame() 
time_lst = pd.date_range(start='1/1/2020', periods=9, freq='M').tolist()
final_data['Period'] = time_lst

In [6]:
# Data cleaning helper function
def DataCleaning(dataname, colname):
    """
    Requires dataname: a file have only 2 rows of years, columns from "Jan" to "Dec"
    Returns: an updated version of final_data, with <colname>
    """
    # import data
    temp_data = pd.read_csv("ManipulatedData/" + dataname)
    # add subset total pplt only to combine data
    final_data[colname] = pd.Series(temp_data.loc[1, "Jan":"Sep"].tolist())

## 2.2 Work with Individual Raw Data Sources

### 2.2.1 COVID ###
- imported datasets are manually edited to condense and format for further data cleaning, including:
    - date colunm was manually formated to consistent date
    - turn rest of columns datatype to numbers

In [7]:
# import NY maniputed data
covid_ny = pd.read_csv("ManipulatedData/NYC_covid.csv")
# reformate date types
covid_ny['date_of_interest'] = covid_ny['date_of_interest'].apply(pd.to_datetime, utc=True)
# obtain month and day data to help locate obtaining end-of-month rows
covid_ny['months'] = covid_ny['date_of_interest'].dt.strftime('%m')

# Obtain cumulative monthly data, and adjust rows for subset data
covidNY_sum = covid_ny.groupby('months').sum()   # sum cases for cumulative monthly data
new_row = pd.DataFrame({'CASE_COUNT':0, 'HOSPITALIZED_COUNT':0, 'DEATH_COUNT':0}, index =[0])
covidNY_sum = pd.concat([new_row, covidNY_sum]).reset_index(drop = True)  # add Jan data
covidNY_sum = covidNY_sum.loc[0:8]               # drop October incomplete data

# add subset total counts to combine data
final_data["TotalCasesNY"] = covidNY_sum['CASE_COUNT']
final_data["TotalDeathsNY"] = covidNY_sum['DEATH_COUNT']
final_data["DeathRateNY"] = (final_data['TotalDeathsNY'] / final_data['TotalCasesNY']).fillna(0)

In [8]:
# import LA maniputed data
covid_la = pd.read_csv("ManipulatedData/LA_county_covid.csv")
# reformate date types
covid_la['date_use'] = covid_la['date_use'].apply(pd.to_datetime, utc=True)
# obtain month and day data to help locate obtaining end-of-month rows
covid_la['months'] = covid_la['date_use'].dt.strftime('%m')

# Obtain cumulative monthly data, and adjust rows for subset data
covidLA_sum = covid_la.groupby('months').sum()   # sum cases for cumulative monthly data
covidLA_sum = covidLA_sum[['new_case', 'new_deaths']]  

feb_row = pd.DataFrame({'new_case':147, 'new_deaths':0}, index =[0])
covidLA_sum = pd.concat([feb_row, covidLA_sum]).reset_index(drop = True)  # add Feb data
jan_row = pd.DataFrame({'new_case':0, 'new_deaths':0}, index =[0])
covidLA_sum = pd.concat([jan_row, covidLA_sum]).reset_index(drop = True)  # add Jan data
covidLA_sum = covidLA_sum.loc[0:9]               # drop October incomplete data

# add subset total counts to combine data
final_data["TotalCasesLA"] = covidLA_sum['new_case']
final_data["TotalDeathsLA"] = covidLA_sum['new_deaths']
final_data["DeathRateLA"] = (final_data['TotalDeathsLA'] / final_data['TotalCasesLA']).fillna(0)

### 2.2.2 Jail ###
- imported datasets are manually edited to condense and format for further data cleaning, including:
    - remove historical data prior to 2019
    - remove analysis cells besides data records
    - remove unnames columns
    - turn datatype to numbers

In [9]:
# import NY maniputed data
jail_ny = pd.read_csv("ManipulatedData/NYC_jail_pplt.csv")   
# add subset total pplt only to combine data
final_data["JailPpltNY"] = pd.Series(jail_ny.loc[0, "20-Jan":"20-Aug"].tolist())

# import LA maniputed data
jail_la = pd.read_csv("ManipulatedData/LAcounty_jail_pplt.csv")
# add subset total pplt only to combine data
final_data["JailPpltLA"] = jail_la.loc[4, "January,2020":"Sept, 2020"].tolist()

### 2.2.3 Unemployment ###
- to use helper function, the dataset is manipulated into specified format

In [10]:
# import and clean data by calling helper function
DataCleaning("NYC_unemployment.csv", "UnemplNY")
DataCleaning("LA_unemployment.csv", "UnemplLA")

### 2.2.4 Weekly working Hours ###
- to use helper function, the dataset is manipulated into specified format

In [11]:
# import and clean data by calling helper function
DataCleaning("NYC_weekly_hours.csv", "WorkHrNY")
DataCleaning("LA_weekly_hours.csv", "WorkHrLA")

### 2.2.5 Computer services employee ###
- to use helper function, the dataset is manipulated into specified format

In [12]:
# import and clean data by calling helper function
DataCleaning("NYC_computer.csv", "CompEmplNY")
DataCleaning("LA_computer.csv", "CompEmplLA")

### 2.2.6 Local government employee ###
- to use helper function, the dataset is manipulated into specified format

In [13]:
# import and clean data by calling helper function
DataCleaning("NYC_localgovernment.csv", "GovEmplNY")
DataCleaning("LA_localgovernment.csv", "GovEmplLA")

### 2.2.7 Hospital employee ###
- to use helper function, the dataset is manipulated into specified format

In [14]:
# import and clean data by calling helper function
DataCleaning("NYC_hospitals.csv", "HospEmplNY")
DataCleaning("LA_hospitals.csv", "HospEmplLA")

### 2.2.8 Financial activities employee ###
- to use helper function, the dataset is manipulated into specified format

In [15]:
# import and clean data by calling helper function
DataCleaning("NYC_financialactivities.csv", "FinEmplNY")
DataCleaning("LA_financialactivities.csv", "FinEmplLA")

### 2.2.9 Educational services employee ###
- to use helper function, the dataset is manipulated into specified format

In [16]:
# import and clean data by calling helper function
DataCleaning("NYC_educationalservices.csv", "EduEmplNY")
DataCleaning("LA_educationalservices.csv", "EduEmplLA")

### 2.2.10 Crime ###
- imported datasets are manually edited to condense and format for further data cleaning, including:
    - data is shrinked by unused rows and columns to reduce file size
    - manipulate datatype to be consistent

In [17]:
crime_ny = pd.read_csv("ManipulatedData/NYC_crime.csv")
# reformate date types
crime_ny['CMPLNT_FR_DT'] = crime_ny['CMPLNT_FR_DT'].apply(pd.to_datetime, utc=True)
# obtain month and day data to help locate obtaining month rows
crime_ny['months'] = crime_ny['CMPLNT_FR_DT'].dt.strftime('%m')
# obtain monthly counts
crimeNY_sum = crime_ny.groupby('months').count()
crimeNY_sum = crimeNY_sum['CMPLNT_FR_DT']
final_data["CrimeNY"] = pd.Series(crimeNY_sum.tolist())

In [18]:
crime_la = pd.read_csv("ManipulatedData/LA_crime.csv")
# reformate date types
crime_la['Date Rptd'] = crime_la['Date Rptd'].apply(pd.to_datetime, utc=True)
# obtain month and day data to help locate obtaining month rows
crime_la['months'] = crime_la['Date Rptd'].dt.strftime('%m')
# obtain monthly counts
crimeLA_sum = crime_la.groupby('months').count()
final_data["CrimeLA"] = pd.Series(crimeLA_sum['Date Rptd'].tolist())

In [19]:
final_data

,Period,TotalCasesNY,TotalDeathsNY,DeathRateNY,TotalCasesLA,TotalDeathsLA,DeathRateLA,JailPpltNY,JailPpltLA,UnemplNY,...,GovEmplNY,GovEmplLA,HospEmplNY,HospEmplLA,FinEmplNY,FinEmplLA,EduEmplNY,EduEmplLA,CrimeNY,CrimeLA
0,2020-01-31,0,0,0.000000,0,0,0.000000,5544.0,6035.0,3.5,...,489.6,578.0,168.4,152.3,475.3,342.8,246.3,173.6,38266,15991
1,2020-02-29,65191,2191,0.033609,147,0,0.000000,5356.0,5960.0,3.4,...,499.3,588.6,168.2,152.6,477.7,345.8,260.7,182.1,35192,16483
2,2020-03-31,109312,12734,0.116492,6478,95,0.014665,5195.0,6015.0,4.1,...,503.8,593.6,167.7,153.5,462.0,346.9,260.2,179.5,32237,15540
3,2020-04-30,28460,2830,0.099438,22023,1107,0.050266,3973.0,5798.0,15.0,...,486.4,572.5,165.5,155.8,455.6,329.5,226.7,166.8,24635,15166
4,2020-05-31,10882,762,0.070024,30158,1235,0.040951,3859.0,5758.0,18.3,...,477.7,558.6,164.1,154.9,453.8,330.1,221.8,162.7,31815,16314
5,2020-06-30,9826,348,0.035416,52148,983,0.018850,3878.0,5620.0,20.3,...,472.8,555.0,165.3,155.1,453.9,331.4,212.4,157.3,32178,16969
6,2020-07-31,7469,148,0.019815,79700,1192,0.014956,3843.0,5223.0,19.9,...,419.9,501.9,166.1,154.1,456.1,337.4,208.9,147.9,34941,16913
7,2020-08-31,10354,135,0.013038,41481,1080,0.026036,3972.0,5053.0,16.0,...,447.8,502.0,165.5,154.1,460.8,335.8,208.0,148.1,36498,16573
8,2020-09-30,16791,162,0.009648,27127,618,0.022782,NaN,4916.0,13.9,...,492.5,544.3,165.6,153.4,456.4,332.6,209.4,151.3,33147,14230


In [20]:
final_data.to_csv("finaldata.csv", sep=",")

# 3. Prepare COVID & Crime Daily Data

In [21]:
# initialize final dataset
crime_covid = pd.DataFrame() 
time_lst = pd.date_range(start='3/1/2020', end="9/28/2020", freq='D').tolist()
crime_covid['Period'] = time_lst

In [22]:
crime_ny = pd.read_csv("ManipulatedData/NYC_crime.csv")
# reformate date types
crime_ny['CMPLNT_FR_DT'] = crime_ny['CMPLNT_FR_DT'].apply(pd.to_datetime, utc=True)

# obtain month and day data to help locate obtaining month rows
crime_ny['days'] = crime_ny['CMPLNT_FR_DT'].dt.strftime('%m%d')
crime_ny.head()
# obtain monthly counts
crimeNY_sum = crime_ny.groupby('days').count()
crimeNY_sum = crimeNY_sum['CMPLNT_FR_DT']

crime_covid["CrimeNY"] = pd.Series(crimeNY_sum.tolist())

In [23]:
crime_la = pd.read_csv("ManipulatedData/LA_crime.csv")
# reformate date types
crime_la['Date Rptd'] = crime_la['Date Rptd'].apply(pd.to_datetime, utc=True)

# obtain month and day data to help locate obtaining month rows
crime_la['days'] = crime_la['Date Rptd'].dt.strftime('%m%d')

# obtain monthly counts
crimeLA_sum = crime_la.groupby('days').count()
crime_covid["CrimeLA"] = pd.Series(crimeLA_sum['Date Rptd'].tolist())

In [24]:
# import data
ny_covid = pd.read_csv("ManipulatedData/NYC_covid.csv")
# data cleaning & add additional columns
ny_covid['date_of_interest'] = ny_covid['date_of_interest'].apply(pd.to_datetime, utc=True)
crime_covid["CaseNY"] = pd.Series(ny_covid['CASE_COUNT'].tolist())
crime_covid["DeathNY"] = pd.Series(ny_covid['DEATH_COUNT'].tolist())

In [25]:
# import data
la_covid = pd.read_csv("ManipulatedData/LA_county_covid.csv")
# data cleaning & add additional columns
la_covid['date_of_interest'] = la_covid['date_use'].apply(pd.to_datetime, utc=True)
crime_covid["CaseLA"] = pd.Series(la_covid['new_case'].tolist())
crime_covid["DeathLA"] = pd.Series(la_covid['new_deaths'].tolist())

In [26]:
crime_covid

,Period,CrimeNY,CrimeLA,CaseNY,DeathNY,CaseLA,DeathLA
0,2020-03-01,1654,350,0,0,25,0
1,2020-03-02,1279,393,0,0,19,0
2,2020-03-03,1255,396,1,0,13,0
3,2020-03-04,1105,425,5,0,19,0
4,2020-03-05,1118,415,3,0,33,0
...,...,...,...,...,...,...,...
207,2020-09-24,1120,512,468,2,1188,19
208,2020-09-25,1192,564,454,4,1072,17
209,2020-09-26,1208,587,279,6,729,20
210,2020-09-27,1153,566,382,2,475,18


In [27]:
crime_covid.to_csv("crimecovid.csv", sep=",")